<a href="https://colab.research.google.com/github/silviolima07/Cluster_cidades/blob/main/Cluster_cidades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***IDENTIFICAR GRUPOS DE CIDADES PARA RECEBER UMA INDUSTRIA DE BEBIDAS***
# Uma estrategia usada foi usar apenas as colunas mais relevantes dentro do contexto do problema.
# Foram identificados no dataset, grupos de cidades com renda per capita semelhante e grupos de cidades com distancia ate a capital, semelhantes.
# Dessa forma foram identificados grupos de cidades que apresentam a mesma faixa de renda e mesma faixa de distancia ate a capital.
# Outra estrategia adotada foi remover as colunas categoricas e aplicar o algoritmo Kmeans para identificar grupos de cidades no dataset.
# Foram identificados 3 grupos de cidades em cada uma das estrategias.
# O cluster de renda foi caracterizados pelo valor medio. Assim, podemos identificar a renda media das cidades de cada grupo.
# O mesmo procedimento foi feito para o cluster de distancia ate a capital.

# Tarefas:
- Classifique os municípios brasileiros com base nas informações disponíveis (utilize um algoritmo de  cluster, se julgar necessário). Caracterize os grupos de municípios criado da melhor maneira possível. 

- Quais grupos de municípios deveriam ser a porta de entrada para empresa no país? Por quê?

- Cálculo da probabilidade de um determinado município pertencer a um dos grupos 


In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#
import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing, cluster
import yellowbrick
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

from sklearn.preprocessing import MinMaxScaler

from sklearn import preprocessing, metrics

%matplotlib inline


In [2]:
def hist_plot(df,var,bins=20):
  return df.hist(column = var, figsize=(7,6), bins=bins);

In [3]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [4]:
def number_of_cluster(coluna):
  sse={}
  df_cluster = df[[coluna]]
  for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(df_cluster)
    df_cluster["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_
    
  plt.figure(figsize=(10, 5))
  plt.plot(list(sse.keys()), list(sse.values()))
  plt.title("Number of Cluster by Column\n"+coluna)
  plt.xlabel("Number of cluster")
  plt.show()

In [5]:
def create_cluster(df,coluna, nova_coluna):
  #building 3 clusters for Frequency and adding it to dataframe
  kmeans = KMeans(n_clusters=3)
  kmeans.fit(df[[coluna]])
  df[nova_coluna] = kmeans.predict(df[[coluna]])
  df = order_cluster(nova_coluna, coluna,df,False)

In [6]:
def xgb_model (X_treino, y_treino, cv=10):
  cv_scores = cross_val_score(xgb, X_treino, y_treino, cv=cv, scoring='accuracy')
  print("Média dos", cv, "testes:", np.round(cv_scores.mean(),4)) 

In [7]:
df = pd.read_csv('/content/drive/MyDrive/base_municipio.csv', encoding='latin-1')

FileNotFoundError: ignored

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

## Manter apenas os dados referentes ao ano 2000

In [ ]:
ano_1991 = [
       'População de 25 anos ou mais de idade, 1991',
       'População de 65 anos ou mais de idade, 1991',
       'População total, 1991']

In [ ]:
df.drop(columns=ano_1991, inplace=True)

In [ ]:
df.drop(columns=['Código'], inplace=True)

In [ ]:
df

In [ ]:
df.Município = df.Município.str.strip()

## Distribuição da renda per capita pelas cidades

In [ ]:
sns.set(palette='muted', color_codes=True, style='white')
fig, ax = plt.subplots(figsize=(12, 6))
sns.despine(left=True)
sns.distplot(df['Renda per Capita, 2000'], bins=30)
plt.show();

# Distribuição da distância da cidade até a capital.

---



In [ ]:
sns.set(palette='muted', color_codes=True, style='white')
fig, ax = plt.subplots(figsize=(12, 6))
sns.despine(left=True)
sns.distplot(df['Distância à capital (km)'], bins=30)
plt.show();

## Quantidade de clusters para Renda per Capita

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12))
X = df['Renda per Capita, 2000'].values.reshape(-1,1)

visualizer.fit(X)    # Fit the data to the visualizer
visualizer.poof() ;   # Draw/show/poof the data

In [ ]:
create_cluster(df,'Renda per Capita, 2000', 'Cluster_Renda' )

In [ ]:
df[['Renda per Capita, 2000', 'Cluster_Renda']]


## Quantidade de clusters para Distancia a capital

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12))
X = df['Distância à capital (km)'].values.reshape(-1,1)

visualizer.fit(X)    # Fit the data to the visualizer
visualizer.poof();    # Draw/show/poof the data

In [ ]:
create_cluster(df,'Distância à capital (km)', 'Cluster_Distancia' )

In [ ]:
df[['Distância à capital (km)', 'Cluster_Distancia']]

## Totais de cidades em cada grupo

In [ ]:
df.Cluster_Renda.value_counts()

In [ ]:
df.Cluster_Distancia.value_counts()

In [ ]:
df_all_cols = df

In [ ]:
df_all_cols[['Município', 'Cluster_Renda', 'Cluster_Distancia']]


#Aplicando kmeans a todas variaveis numericas

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/base_municipio.csv', encoding='latin-1')

In [ ]:
df.drop(columns=['Código'], inplace=True)

In [ ]:
df2 = df.copy()

## Manter apenas as variaveis numericas

In [ ]:
df2.drop(['Município', 'UF'], axis=1, inplace=True)

In [ ]:
df2.head()

In [ ]:
X = df2

In [ ]:
X.head()

In [ ]:
cols = X.columns

## Normalizando as escala de valores

In [ ]:
ms = MinMaxScaler()

X = ms.fit_transform(X)

In [ ]:
X = pd.DataFrame(X, columns=[cols])

In [ ]:
X.head()

## Utilizando metodos da lib Yellowbrick para identificar o numero de cluster

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12))
X = X

visualizer.fit(X)    # Fit the data to the visualizer
visualizer.poof();    # Draw/show/poof the data

## Criar o total de clusters informado no passo anterior

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0) 

kmeans.fit(X)

In [ ]:
df2['3_clusters'] = kmeans.labels_

In [ ]:
df2['3_clusters']

In [ ]:
df2.columns

In [ ]:
temp2 = df2['3_clusters']

In [ ]:
df3 = pd.concat([df_all_cols,temp2], axis=1)
df3

In [ ]:
df3['3_clusters'].value_counts()

In [ ]:
df4 = df3[['Município', 'UF', 'Renda per Capita, 2000', 'Distância à capital (km)', 'Cluster_Renda', 'Cluster_Distancia', '3_clusters']]

In [ ]:

df4

# Classifique os municípios brasileiros com base nas informações disponíveis (utilize um algoritmo de cluster, se julgar necessário). 
# Caracterize os grupos de municípios criado da melhor maneira possível.

## ***Foram usados os cluster criados a partir da Renda per capita e da Distancia ate a capital.***

## ***Renda media das cidades no grupo***



In [ ]:
renda_g2 = df4.loc[df4['Cluster_Renda']==2]['Renda per Capita, 2000']
renda_g1 = df4.loc[df4['Cluster_Renda']==1]['Renda per Capita, 2000']
renda_g0 = df4.loc[df4['Cluster_Renda']==0]['Renda per Capita, 2000']
#
print('Renda media grupo 0: R$',round(renda_g0.mean(),2))
renda_g00 = 'Renda media de R$'+ str(round(renda_g0.mean(),2))
print('Renda media grupo 1: R$',round(renda_g1.mean(),2))
renda_g11 = 'Renda media de R$'+ str(round(renda_g1.mean(),2))
print('Renda media grupo 2: R$',round(renda_g2.mean(),2))
renda_g22 = 'Renda media de R$'+ str(round(renda_g2.mean(),2))
#
# Labels

renda = np.where(df4['Cluster_Renda'] == 0, renda_g00,
          np.where(df4['Cluster_Renda'] == 1, renda_g11,
                     np.where(df4["Cluster_Renda"] == 2, renda_g22, renda_g00)))

In [ ]:
temp2 = renda_g2.mean()
temp1 = renda_g1.mean()
temp0 = renda_g0.mean()

In [ ]:
if (temp2 > temp1) & (temp2 > temp0):
  max_renda_media = 2
elif (temp1 > temp2) & (temp1 > temp0):
  max_renda_media = 1
elif (temp0 > temp1) & (temp0 > temp2):
  max_renda_media = 0    

In [ ]:
df4['Label_Renda'] = renda

In [ ]:
df4[['Cluster_Renda', 'Label_Renda']]

## ***Distancia media ate a capital***

In [ ]:
distancia_g2 = df4.loc[df4['Cluster_Distancia']==2]['Distância à capital (km)']
distancia_g1 = df4.loc[df4['Cluster_Distancia']==1]['Distância à capital (km)']
distancia_g0 = df4.loc[df4['Cluster_Distancia']==0]['Distância à capital (km)']
#
print('Distância media grupo 0: km',round(distancia_g0.mean(),2))
distancia_g00 = 'Distancia media de '+ str(round(distancia_g0.mean(),2))+' km'
print('Distancia media grupo 1: km',round(distancia_g1.mean(),2))
distancia_g11 = 'Distancia media de '+ str(round(distancia_g1.mean(),2))+' km'
print('Distancia media grupo 2: km',round(distancia_g2.mean(),2))
distancia_g22 = 'Distancia media de '+ str(round(distancia_g2.mean(),2))+' km'
#
# Labels

distancia = np.where(df4['Cluster_Distancia'] == 0, distancia_g00,
          np.where(df4['Cluster_Distancia'] == 1, distancia_g11,
                     np.where(df4["Cluster_Distancia"] == 2, distancia_g22, distancia_g00)))

In [ ]:
temp2 = distancia_g2.mean()
temp1 = distancia_g1.mean()
temp0 = distancia_g0.mean()

In [ ]:
if (temp2 < temp1) & (temp2 < temp0):
  min_distancia_media = 2
elif (temp1 < temp2) & (temp1 < temp0):
  min_distancia_media = 1
elif (temp0 < temp1) & (temp0 < temp2):
  min_distancia_media = 0    

In [ ]:
min_distancia_media

In [ ]:
df4['Label_Distancia'] = distancia

In [ ]:
df4[['Cluster_Distancia', 'Label_Distancia']]

In [ ]:
df4.columns

## Cidades caracterizadas pela renda media e distancia ate a capital

In [ ]:
df4[['Município', 'UF',
       'Cluster_Renda', 'Label_Renda','Cluster_Distancia', 
       'Label_Distancia', '3_clusters']]

### Quais grupos de municípios deveriam ser a porta de entrada para empresa no país? 
### Por quê?

# Estrategia:
#Identificar as cidades com maior renda media e menor distancia ate a capital

In [ ]:
df4

In [ ]:
df4.loc[(df4['Cluster_Renda']== max_renda_media) & (df4['Cluster_Distancia']== min_distancia_media)][['Município', 'UF',
       'Cluster_Renda', 'Label_Renda','Cluster_Distancia', 
       'Label_Distancia', '3_clusters']]

# A analise feita acima, selecionou as cidades com maior renda media e menor distancia media ate a capital. Baseando-se apenas em renda e distancia, essas cidades seriam as apontadas como porta de entrada.
# Maior poder aquisitivo impacta no consumo e por consequencia na produção.
# Menor distancia com a capital, pode significar menor custo para distribuição em grandes centros.

# 

### Qual a probabilidade de um determinado município pertencer a um dos grupos?

In [ ]:
df2

In [ ]:
df2['3_clusters'].value_counts()

In [ ]:
plt.figure(figsize=(16,8))

plt.subplot(233)
sns.countplot(y='3_clusters',data=df2)
plt.gca().xaxis.tick_bottom()
plt.title('Total de cidades por cluster');

####  A probabilidade de um municipio pertencer a um dos grupos encontrados é a quantidade de cidades no grupo dividido pela quantidade total de cidades.

In [ ]:
prob_grupo1 = (2348 / 5507)*100

In [ ]:
from google.colab import drive
drive.mount('/content/drive')